In [408]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt

import csv file

In [409]:
df=pd.read_csv('bank-full.csv',delimiter=";")

In [ ]:
df.columns
df

Select only the features from above.

In [ ]:
df = df[['age', 'job', 'marital', 'education', 'balance', 'housing', 'contact', 
                    'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']]


df.columns



Check if the missing values are presented in the features.

In [ ]:
df.isnull().sum()

Question 1
What is the most frequent observation (mode) for the column education?

In [ ]:
df_education_mode = df[["education"]].mode()
df_education_mode

Question 2
Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

In [ ]:
df.dtypes
df_numerical = df[['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']]
df_numerical

In [ ]:
df

In [ ]:
# Creating the correlation matrix for the numerical features
df_numerical_corr=df_numerical.corr()
df_numerical_corr

# Finding the two features with the highest correlation
max_corr_value = df_numerical_corr.abs().unstack().sort_values()
# Exclude the correlation of the same features (diagonal)
max_corr_value = max_corr_value[max_corr_value < 1]

# Getting the two features with the highest correlation
top_correlation = max_corr_value.idxmax(), max_corr_value.max()

top_correlation


In [ ]:
df

Target encoding
Now we want to encode the y variable.
Let's replace the values yes/no with 1/0.

In [ ]:
df.y = (df.y == 'yes').astype(int)

In [ ]:
df


Split the data
Split your data in train/val/test sets with 60%/20%/20% distribution.
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
Make sure that the target value y is not in your dataframe.

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into 60% train, 20% validation, and 20% test
# First, we split into train (80%) and temp (20%) with a random_state of 42 for reproducibility
train_test_split (df, test_size=0.2,random_state=1)
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
len(df_train_full),len(df_test)
# Now, we split the temp set into validation (20%) and train (60%)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)
len(df_train),len(df_test),len(df_val)

In [421]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values
#drop y
#del df_train['y']
#del df_val['y']
#del df_test['y']

Question 3
Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
Round the scores to 2 decimals using round(score, 2).

In [ ]:
df

In [ ]:


global_mean = df_train_full.y.mean()
round(global_mean, 2)



In [ ]:
#
married_mean = df_train_full[df_train_full.marital == 'married'].y.mean()
print('marital == married:', round(married_mean, 2))

notmarried_mean = df_train_full[df_train_full.marital == 'married'].y.mean()
print('marital == no:  ', round(notmarried_mean, 2))



In [ ]:
notmarried_mean / global_mean

married_mean / global_mean

df_group = df_train_full.groupby(by="marital").y.agg(['mean'])
df_group['diff'] = df_group['mean'] - global_mean
df_group['risk'] = df_group['mean'] / global_mean
df_group

In [426]:
categorical = ['job', 'marital', 'education',  'housing',
                 'contact', 'month',  'poutcome', 'y']
numerical = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous','y']

In [ ]:
df_train_full

In [ ]:
from IPython.display import display

for col in categorical:
    df_group = df_train_full.groupby(by=col).y.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['risk'] = df_group['mean'] / global_mean
    display(df_group)

            
        


In [ ]:
from sklearn.metrics import mutual_info_score

def calculate_mi(series):
    return mutual_info_score(series, df_train_full.y)

df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')


display(df_mi.head())
display(df_mi.tail())




In [430]:
def calculate_mi(series):
    return mutual_info_score(series, df_train_full.y)

Question 4

In [ ]:
df_train_full[numerical].corrwith(df_train_full.y).to_frame('correlation')
df_train_full.groupby(by='y')[numerical].mean()

In [ ]:
from sklearn.feature_extraction import DictVectorizer
train_dict = df_train[categorical + numerical].to_dict(orient='records')
train_dict[0]
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)
X_train = dv.transform(train_dict)
X_train.shape


In [434]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(X_train, y_train)

In [442]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)
model.predict_proba(X_val)
y_pred = model.predict_proba(X_val)[:, 1]
y_pred
y = y_pred > 0.5
(y_val == y).mean()

C:\Users\angel_buvawpg\AppData\Local\Temp\ipykernel_151644\1817052294.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  val_dict = df_val[categorical + numerical].to_dict(orient='records')


1.0

Question 5
Let's find the least useful feature using the feature elimination technique.
Train a model with all these features (using the same parameters as in Q4).
Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
Which of following feature has the smallest difference?


In [449]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

def train_and_evaluate(X_train, X_val, y_train, y_val):
   
# Train logistic regression model
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fitmodel.fit(X_train, y_train)
    
    # Predict on validation set and calculate accuracy
y_val_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
    


# Train the model with all features and record accuracy
original_accuracy = train_and_evaluate(X_train, X_val, y_train, y_val)

# List of features to test
features_to_test = ['age', 'balance', 'marital', 'previous']

# Dictionary to store accuracy differences
accuracy_differences = {}

# Test excluding each feature
for feature in features_to_test:
    # Drop the feature from the dataset
    X_train_excluded = X_train.drop(columns=[feature])
    X_val_excluded = X_val.drop(columns=[feature])
    
    # Train and evaluate the model without the feature
    accuracy_without_feature = train_and_evaluate(X_train_excluded, X_val_excluded, y_train, y_val)
    
    # Calculate the accuracy difference
    accuracy_difference = round(original_accuracy - accuracy_without_feature, 4)
    accuracy_differences[feature] = accuracy_difference

# Output the feature with the smallest difference
least_impactful_feature = min(accuracy_differences, key=accuracy_differences.get)

print(f"Feature with the smallest difference: {least_impactful_feature}")


AttributeError: 'LogisticRegression' object has no attribute 'fitmodel'